In [ ]:
!pip install -qU langchain-core openai supabase

In [ ]:
import os
import openai
from supabase import create_client, Client
from google.colab import userdata
from langchain_core.embeddings import DeterministicFakeEmbedding

In [ ]:
url = userdata.get("SUPABASE_URL")
key = userdata.get("SUPABASE_KEY")
api_key: str = userdata.get("OPENAI_API_KEY")
openai.api_key = api_key
supabase: Client = create_client(url, key)

In [ ]:
embeddings = DeterministicFakeEmbedding(size=1536)

In [ ]:
def get_context(question: str) -> str:
    # Embed the user's question
    question_embedding = embeddings.embed_query(question)

    results = []
    if "customer" in question.lower():
        query = supabase.rpc("find_related_customer", {'question_vector': question_embedding}).execute()
    elif "product" in question.lower():
        query = supabase.rpc("find_related_products", {'question_vector': question_embedding}).execute()
    elif "invoice" in question.lower():
        query = supabase.rpc("find_related_invoices", {'question_vector': question_embedding}).execute()
    else:
        return "No relevant context found for the given question."

    for item in query.data:
        results.append(item)
    return results

In [ ]:
def get_response(question: str):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions \
                                          about the customers, products, and invoices provided to \
                                          you in the context. Use only the provided context to answer \
                                          questions. If the information isn't in the context, say so."},
            {"role": "user", "content": f"Question: {question}\n\nContext:\n{get_context(question)}"}
        ],
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

In [ ]:
question = "Is there a customer named Justin Chau? If so, show me his information"
answer = get_response(question)
print("Answer:", answer)